In [5]:
import sys
try:
    from typing import BinaryIO, TextIO
except ImportError:
    pass

# Tạo giả module typing.io và typing.re để tránh lỗi
if 'typing.io' not in sys.modules:
    import types
    sys.modules['typing.io'] = types.ModuleType('typing.io')
    sys.modules['typing.io'].BinaryIO = bytes
    sys.modules['typing.io'].TextIO = str

if 'typing.re' not in sys.modules:
    sys.modules['typing.re'] = types.ModuleType('typing.re')

In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, date_format, avg, dayofweek
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, dayofweek, month
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [8]:
# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("WeatherPredictionSpark") \
    .config("spark.ui.port", "4050") \
    .config("spark.ui.enabled", "true") \
    .getOrCreate()

# Đọc file CSV
# 'inferSchema=True' để Spark tự động xác định kiểu dữ liệu
# 'header=True' để sử dụng hàng đầu tiên làm tên cột
df = spark.read.csv(r'D:\jena_climate_processed.csv', inferSchema=True, header=True)

# Hiển thị 5 hàng đầu tiên và schema để kiểm tra
print("Schema:")
df.printSchema()
print("Dữ liệu (5 hàng đầu):")
df.show(5)

Schema:
root
 |-- Date Time: timestamp (nullable = true)
 |-- p (mbar): double (nullable = true)
 |-- T (degC): double (nullable = true)
 |-- Tpot (K): double (nullable = true)
 |-- Tdew (degC): double (nullable = true)
 |-- rh (%): double (nullable = true)
 |-- VPmax (mbar): double (nullable = true)
 |-- VPact (mbar): double (nullable = true)
 |-- VPdef (mbar): double (nullable = true)
 |-- sh (g/kg): double (nullable = true)
 |-- H2OC (mmol/mol): double (nullable = true)
 |-- rho (g/m**3): double (nullable = true)
 |-- wv (m/s): double (nullable = true)
 |-- max. wv (m/s): double (nullable = true)
 |-- wd (deg): double (nullable = true)
 |-- hour_sin: double (nullable = true)
 |-- hour_cos: double (nullable = true)
 |-- day_sin: double (nullable = true)
 |-- day_cos: double (nullable = true)
 |-- month_sin: double (nullable = true)
 |-- month_cos: double (nullable = true)

Dữ liệu (5 hàng đầu):
+-------------------+--------+--------+--------+-----------+------+------------+----------

# Tiền xử lý dữ liệu với pyspark

In [9]:
# Chuyển đổi cột 'Date Time' sang Timestamp và tạo cột 'Date'
df_processed = df.withColumn("Timestamp", to_timestamp(col("Date Time"), "yyyy-MM-dd HH:mm:ss")) \
                 .withColumn("Date", date_format(col("Timestamp"), "yyyy-MM-dd"))

In [10]:
# Tính toán các giá trị trung bình hằng ngày
daily_df = df_processed.groupBy("Date").agg(
    avg(col("T (degC)")).alias("Avg_Temp"),
    avg(col("p (mbar)")).alias("Avg_Pressure"),
    avg(col("rh (%)")).alias("Avg_Humidity"),
    avg(col("wv (m/s)")).alias("Avg_WindSpeed"),
).orderBy("Date")

In [11]:
daily_df.show(5)
daily_df.printSchema()

+----------+-------------------+-----------------+-----------------+------------------+
|      Date|           Avg_Temp|     Avg_Pressure|     Avg_Humidity|     Avg_WindSpeed|
+----------+-------------------+-----------------+-----------------+------------------+
|2009-01-01|  -6.81062937062937|999.1455944055942|91.08601398601404|0.7786013986013983|
|2009-01-02|-3.7281944444444455|999.6006250000005|92.08680555555554|1.4195138888888885|
|2009-01-03|  -5.27173611111111|998.5486111111109|76.45805555555556| 1.250902777777778|
|2009-01-04|-1.3752083333333327|988.5106944444441|89.41736111111112|1.7204166666666663|
|2009-01-05| -4.867152777777779|990.4056944444445| 86.2604166666667|3.8002777777777776|
+----------+-------------------+-----------------+-----------------+------------------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- Avg_Temp: double (nullable = true)
 |-- Avg_Pressure: double (nullable = true)
 |-- Avg_Humidity: double (nullable = true)
 |-- Avg_WindS

# Xây dựng Features (Lag Features)

In [12]:
# Tạo một cửa sổ được sắp xếp theo 'Date'
window_spec = Window.orderBy("Date")

In [13]:
# Tạo các lag feature cho nhiệt độ (T-1, T-2 ngày)
# T-1: Nhiệt độ trung bình ngày hôm trước (Lag=1)
# T-7: Nhiệt độ trung bình 7 ngày trước (Lag=7)
daily_df_features = daily_df.withColumn("Temp_Lag_1", lag(col("Avg_Temp"), 1).over(window_spec)) \
                            .withColumn("Temp_Lag_7", lag(col("Avg_Temp"), 7).over(window_spec))

In [14]:
# Thêm các features dựa trên thời gian (ví dụ: ngày trong tuần, tháng)
daily_df_features = daily_df_features.withColumn("DayOfWeek", dayofweek(col("Date"))) \
                                     .withColumn("Month", month(col("Date")))

In [15]:
# Loại bỏ các hàng bị thiếu (NaN) do việc tạo Lag Features (các hàng đầu tiên)
final_df = daily_df_features.na.drop()
final_df.show(5)

+----------+-------------------+------------------+-----------------+------------------+-------------------+-------------------+---------+-----+
|      Date|           Avg_Temp|      Avg_Pressure|     Avg_Humidity|     Avg_WindSpeed|         Temp_Lag_1|         Temp_Lag_7|DayOfWeek|Month|
+----------+-------------------+------------------+-----------------+------------------+-------------------+-------------------+---------+-----+
|2009-01-08| -9.609166666666663|1002.9696527777776|87.58263888888888|0.8702083333333337|-15.734374999999996|  -6.81062937062937|        5|    1|
|2009-01-09|-12.301458333333336|1003.1133333333341|83.97624999999994|0.6638194444444447| -9.609166666666663|-3.7281944444444455|        6|    1|
|2009-01-10|-10.728263888888897|1002.9675694444447|82.34083333333331|0.6664583333333334|-12.301458333333336|  -5.27173611111111|        7|    1|
|2009-01-11| -8.681666666666665|1004.0074999999999|83.40930555555558|1.2350000000000003|-10.728263888888897|-1.3752083333333327|  

# Vector hóa Features và Chia tập Dữ liệu

In [16]:
# Xác định các cột Features (Đầu vào) và Label (Đầu ra)
# Features: Lag Features + Thời gian + Các yếu tố thời tiết khác
feature_cols = ["Temp_Lag_1", "Temp_Lag_7", "DayOfWeek", "Month", "Avg_Pressure", "Avg_Humidity", "Avg_WindSpeed"]
label_col = "Avg_Temp" # Đây là cộtmuốn dự đoán

In [17]:
# VectorAssembler kết hợp các cột features thành một cột vector duy nhất
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
model_df = assembler.transform(final_df)

In [18]:
# Đổi tên cột mục tiêu thành 'label' theo quy ước của MLlib
model_df = model_df.withColumnRenamed(label_col, "label")

In [19]:
# Chia dữ liệu: 80% Train, 20% Test
(train_df, test_df) = model_df.randomSplit([0.8, 0.2], seed=42)

In [20]:
print(f"Kích thước tập Train: {train_df.count()}")
print(f"Kích thước tập Test: {test_df.count()}")

Kích thước tập Train: 2382
Kích thước tập Test: 532


# Huấn luyện mô hình

In [21]:
lr = LinearRegression(featuresCol="features", labelCol="label")

In [22]:
lr_model = lr.fit(train_df)

In [23]:
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

Coefficients: [0.8453146054303174,0.06228900943321805,-0.003529390751808878,0.05772320668759937,-0.017302915753453267,-0.08002720260670083,0.0005074871173320281]
Intercept: 23.664384592033382


# Đánh giá mô hình

In [24]:
# Dự đoán trên tập Test
predictions = lr_model.transform(test_df)

In [25]:
# Đánh giá mô hình
evaluator = RegressionEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="rmse" # Root Mean Squared Error (Sai số vuông trung bình)
)

In [26]:
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

In [27]:
print(f"Root Mean Squared Error (RMSE) trên tập Test: {rmse:.2f} degC")
print(f"R-squared (R2) trên tập Test: {r2:.2f}")

Root Mean Squared Error (RMSE) trên tập Test: 2.20 degC
R-squared (R2) trên tập Test: 0.92


In [28]:
# Hiển thị kết quả dự đoán (5 hàng)
predictions.select("Date", "label", "prediction").show(5)

+----------+-------------------+-------------------+
|      Date|              label|         prediction|
+----------+-------------------+-------------------+
|2009-01-10|-10.728263888888897|-10.973003244651135|
|2009-01-14|  -4.68777777777778| -2.230391268036474|
|2009-01-16|-2.8157638888888896|-1.7998342265525586|
|2009-01-21|-1.5480555555555557|  1.722054479464962|
|2009-01-27|-1.3442361111111105|-3.8506173513223807|
+----------+-------------------+-------------------+
only showing top 5 rows



In [29]:
rmse_scores = []
r2_scores = []

print("Empty lists 'rmse_scores' and 'r2_scores' initialized.")

Empty lists 'rmse_scores' and 'r2_scores' initialized.


In [30]:
num_iterations = 10

for i in range(num_iterations):
    print(f"\n Bootstrap Iteration {i+1}/{num_iterations}")

    # 2. Tạo một mẫu bootstrap từ model_df bằng cách lấy mẫu có thay thế
    # Sử dụng seed khác nhau cho mỗi lần lặp để đảm bảo các mẫu khác nhau
    bootstrap_sample = model_df.sample(withReplacement=True, fraction=1.0, seed=42 + i)

    # 3. Chia mẫu bootstrap thành tập huấn luyện và kiểm tra
    (train_bootstrap_df, test_bootstrap_df) = bootstrap_sample.randomSplit([0.8, 0.2], seed=42 + i)

    # 4. Khởi tạo mô hình LinearRegression
    lr_bootstrap = LinearRegression(featuresCol="features", labelCol="label")

    # 5. Huấn luyện mô hình LinearRegression trên dữ liệu huấn luyện bootstrap
    lr_bootstrap_model = lr_bootstrap.fit(train_bootstrap_df)

    # 6. Thực hiện dự đoán trên dữ liệu kiểm tra bootstrap
    predictions_bootstrap = lr_bootstrap_model.transform(test_bootstrap_df)

    # 7. Khởi tạo RegressionEvaluator
    evaluator_bootstrap = RegressionEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="rmse"
    )

    # 8. Tính toán RMSE và thêm vào danh sách
    rmse_bootstrap = evaluator_bootstrap.evaluate(predictions_bootstrap)
    rmse_scores.append(rmse_bootstrap)

    # 9. Tính toán R-squared và thêm vào danh sách
    r2_bootstrap = evaluator_bootstrap.evaluate(predictions_bootstrap, {evaluator_bootstrap.metricName: "r2"})
    r2_scores.append(r2_bootstrap)

    # 10. In số lần lặp và các giá trị RMSE, R-squared đã tính toán
    print(f"Lần 1 {i+1}: RMSE = {rmse_bootstrap:.2f} degC, R-squared = {r2_bootstrap:.2f}")

print("\nBootstrap evaluation completed.")


 Bootstrap Iteration 1/10
Lần 1 1: RMSE = 2.19 degC, R-squared = 0.92

 Bootstrap Iteration 2/10
Lần 1 2: RMSE = 2.12 degC, R-squared = 0.93

 Bootstrap Iteration 3/10
Lần 1 3: RMSE = 2.99 degC, R-squared = 0.86

 Bootstrap Iteration 4/10
Lần 1 4: RMSE = 2.15 degC, R-squared = 0.92

 Bootstrap Iteration 5/10
Lần 1 5: RMSE = 2.26 degC, R-squared = 0.91

 Bootstrap Iteration 6/10
Lần 1 6: RMSE = 2.14 degC, R-squared = 0.93

 Bootstrap Iteration 7/10
Lần 1 7: RMSE = 2.25 degC, R-squared = 0.92

 Bootstrap Iteration 8/10
Lần 1 8: RMSE = 2.29 degC, R-squared = 0.92

 Bootstrap Iteration 9/10
Lần 1 9: RMSE = 2.25 degC, R-squared = 0.91

 Bootstrap Iteration 10/10
Lần 1 10: RMSE = 2.30 degC, R-squared = 0.90

Bootstrap evaluation completed.


In [31]:
import numpy as np
# Tính toán trung bình và độ lệch chuẩn của RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

# Tính toán trung bình và độ lệch chuẩn của R-squared
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)

print(f"\n=== Bootstrap Evaluation Summary ===")
print(f"Mean RMSE: {mean_rmse:.2f} degC (Std Dev: {std_rmse:.2f})")
print(f"Mean R-squared: {mean_r2:.2f} (Std Dev: {std_r2:.2f})")
print(f"Total iterations: {len(rmse_scores)}")


=== Bootstrap Evaluation Summary ===
Mean RMSE: 2.29 degC (Std Dev: 0.24)
Mean R-squared: 0.91 (Std Dev: 0.02)
Total iterations: 10


# Xây dựng website demo

In [35]:
pip install pyngrok

Note: you may need to restart the kernel to use updated packages.


In [32]:
!pip install streamlit pandas numpy scikit-learn

In [36]:
import os
import subprocess
import time
from pyngrok import ngrok
import numpy as np

# ==================== BƯỚC 1: Tạo file Streamlit app ====================
streamlit_app_content = """
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

# Hệ số và intercept từ mô hình Linear Regression đã huấn luyện
coefficients = np.array([
    0.8453146054303309, 0.0622890094332121, -0.003529390751808062,
    0.057723206687585626, -0.01730291575339897, -0.08002720260669162,
    0.000507487117332345
])
intercept = 23.664384591978987
feature_names = ['Temp_Lag_1', 'Temp_Lag_7', 'DayOfWeek', 'Month', 'Avg_Pressure', 'Avg_Humidity', 'Avg_WindSpeed']

st.set_page_config(page_title="Dự đoán Nhiệt Độ", layout="wide")
st.title("🌡️ Dự đoán nhiệt độ hàng ngày (Spark MLlib Linear Regression)")
st.markdown("---")

st.header("Nhập dữ liệu để dự đoán")
col1, col2, col3 = st.columns(3)

with col1:
    temp_lag_1 = st.number_input("Nhiệt độ trung bình ngày hôm trước (℃)", value=-5.0, step=0.1)
    temp_lag_7 = st.number_input("Nhiệt độ trung bình 7 ngày trước (℃)", value=-3.0, step=0.1)
    day_of_week = st.slider("Ngày trong tuần (1=CN, 7=T7)", 1, 7, 4)

with col2:
    month = st.slider("Tháng", 1, 12, 7)
    avg_pressure = st.number_input("Áp suất trung bình (mbar)", value=1000.0, step=0.1)

with col3:
    avg_humidity = st.number_input("Độ ẩm trung bình (%)", value=80.0, step=0.1)
    avg_windspeed = st.number_input("Tốc độ gió trung bình (m/s)", value=2.0, step=0.1)

st.markdown("---")

prediction = None
if st.button("🔮 Dự đoán nhiệt độ"):
    input_features = np.array([temp_lag_1, temp_lag_7, day_of_week, month,
                               avg_pressure, avg_humidity, avg_windspeed])
    prediction = np.dot(input_features, coefficients) + intercept
    
    st.success(f"Nhiệt độ trung bình dự đoán: **{prediction:.2f} °C**")
    st.balloons()

st.markdown("---")
st.header("Thông tin mô hình")
st.json({"Coefficients": coefficients.tolist(), "Intercept": intercept, "Features": feature_names})

st.markdown("---")
st.header("Trực quan hóa ảnh hưởng của Temp_Lag_1")
temp_range = np.linspace(-20, 10, 50)
preds = []
for t in temp_range:
    features = np.array([t, temp_lag_7, day_of_week, month, avg_pressure, avg_humidity, avg_windspeed])
    preds.append(np.dot(features, coefficients) + intercept)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(temp_range, preds, label="Dự đoán", color="skyblue", linewidth=2)
ax.set_xlabel("Nhiệt độ ngày hôm trước (°C)")
ax.set_ylabel("Nhiệt độ dự đoán (°C)")
ax.set_title("Ảnh hưởng của Temp_Lag_1 đến dự đoán")
ax.grid(True, alpha=0.3)

if prediction is not None:
    ax.plot(temp_lag_1, prediction, "ro", markersize=8, label=f"Dự đoán hiện tại: {prediction:.2f}°C")
    ax.legend()

st.pyplot(fig)
"""

In [37]:
app_file = "app.py"
with open(app_file, "w", encoding="utf-8") as f:
    f.write(streamlit_app_content)

print(f"✅ Đã tạo file Streamlit: {app_file}")

✅ Đã tạo file Streamlit: app.py


In [38]:
# BƯỚC 2: Dừng ngrok cũ (nếu có)
print("🔄 Đóng các tunnel ngrok cũ...")
ngrok.kill()

🔄 Đóng các tunnel ngrok cũ...


In [39]:
# BƯỚC 3: Thiết lập ngrok auth token
# Thay bằng token thật của bạn (lấy từ https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "378ldpQuuOJPzNAJbTHvQY0TcVN_45ZSgqEhSbcDHQagT2JyJ"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# BƯỚC 4: Khởi động Streamlit
print("🚀 Đang khởi động Streamlit...")

process = subprocess.Popen([
    "python", "-m", "streamlit", "run", app_file,
    "--server.port=8501",
    "--server.address=0.0.0.0",
    "--server.headless=true",           # Quan trọng khi chạy background
    "--server.enableCORS=false",
    "--server.enableXsrfProtection=false"
],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

print(f"Streamlit đang chạy với PID: {process.pid}")
print("Truy cập local: http://localhost:8501")
# In log realtime để theo dõi khởi động
print("\n=== Log Streamlit ===")
for _ in range(30):  # In tối đa 30 dòng đầu
    line = process.stdout.readline()
    if not line:
        break
    print(line.rstrip())
    # Nếu thấy dòng báo Streamlit sẵn sàng thì dừng in
    if "Local URL" in line or "Network URL" in line:
        break
# BƯỚC 5: Chờ Streamlit sẵn sàng 
print("\n Chờ Streamlit khởi động hoàn toàn (15 giây)...")
time.sleep(15)

🚀 Đang khởi động Streamlit...
Streamlit đang chạy với PID: 7628
Truy cập local: http://localhost:8501

=== Log Streamlit ===



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

